# Факторный анализ. Метод гланых компонент

### Цели и задачи

1. **Применить факторный анализ (метод главных компонет) к совокупности анализируемых данных. Проанализировать целесообразность использования данного метода анализа на основе специальных критериев и тестовых статистик.**
2. **На основании результатов факторного анализа определить наиболее значимые коэффициенты, сохраняющие наибольший процент дисперсии исходных показателей.**
3. **Рассчитать интегральный показатель кредитоспособности как взвешенную сумму полученных главных компонент. В качестве весов использовать соответствующие величины процентов объясненной дисперсии.**

In [1]:
import os
import sys
import warnings
import pandas as pd
import numpy as np
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity


sys.path.append('../src')
warnings.filterwarnings('ignore')

from factor_analysis_utils import create_factor_summary_df, filter_loadings, run_pca_with_kaiser, run_rotated_factor_analysis, calculate_integral_indicator

In [2]:
file_path = '../data/data_censored.csv'
df = pd.read_csv(file_path)

display(df)

,k1,k2,k4,k5,k6,k7,k9,k10,k11,k13,k14,k15,k18,k19
0,0.000000,0.014084,0.000000,0.797945,0.449936,0.000000,1.000000,0.000000,0.012880,0.062375,0.383869,0.795137,0.002558,0.002752
1,0.245124,0.299176,0.558148,0.910623,0.264507,1.000000,1.000000,0.000000,0.028820,0.082380,1.000000,1.000000,0.003869,0.002987
2,0.000000,0.000000,0.000000,0.764261,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000528
3,0.809119,0.453227,0.853955,0.940561,0.899900,0.962963,1.000000,0.339213,0.024809,0.714604,1.000000,1.000000,0.005724,0.011632
4,0.194844,0.000000,0.444533,0.864509,0.000000,0.000000,1.000000,0.007876,0.000000,0.000000,0.514314,0.625071,0.002160,0.001286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,0.323204,0.142995,0.397917,0.757061,0.041652,0.874880,0.795362,0.355109,0.051856,0.419410,0.829911,1.000000,0.004824,0.011298
2691,0.143047,0.151816,0.157581,0.830148,0.354400,1.000000,1.000000,0.006451,0.066174,0.380569,1.000000,1.000000,0.001887,0.004243
2692,0.113393,0.000000,0.190636,0.808592,0.000000,0.000000,0.836614,0.050279,0.045358,0.714604,1.000000,1.000000,0.002285,0.009967
2693,0.077662,0.000000,0.000000,0.748261,0.494889,0.907597,0.868915,0.033910,0.074567,0.439297,0.944861,1.000000,0.001407,0.003355


In [3]:
X = (df - df.mean()) / df.std()

**Перед применением факторного анализа данные стандартизируются для обеспечения сопоставимости переменных и корректного расчёта корреляционной матрицы.**

## Оценка целесообразности применения факторного анализа

**Целесообразность выполнения факторного анализа определяется наличием существенной корреляции между используемыми переменными**

In [4]:
kmo_all, kmo_model = calculate_kmo(X)
chi_square, p_value = calculate_bartlett_sphericity(X)

print(f"Статистика KMO: {kmo_model:.3f}")
print(f"Критерий сферичности Бартлетта: chi^2 = {chi_square:.1f}, p-value = {p_value:.6f}")

Статистика KMO: 0.673
Критерий сферичности Бартлетта: chi^2 = 20932.4, p-value = 0.000000


**Статистика КМО позволяет проверить, насколько корреляцию между парами переменных можно объяснить другими переменными (факторами).**

**Критерий Бартлетта проверяет нулевую гипотезу об отсутствии корреляций между переменными в генеральной совокупности.**

**Полученные значения статистик свидетельствуют о целесообразности применении факторного анализа к рассматриваемым данным.**

## Применение факторного анализа

**Для определения числа факторов используется критерий Кайзера, согласно которому сохраняются компоненты с собственными значениями > 1.**

In [5]:
pca, n_factors, eigen_values, explained_variance, cumulative_variance = run_pca_with_kaiser(X)

factors = create_factor_summary_df(eigen_values, explained_variance * 100, cumulative_variance * 100)
factors_before_rotation = factors[factors['Собственное значение'] > 1]

print("Факторы до вращения")
display(factors_before_rotation)

Факторы до вращения


,Собственное значение,% объясненной дисперсии,Кумулятивный % объясненной дисперсии
1,4.498162,32.129730,32.129730
2,2.326236,16.615970,48.745700
3,1.415678,10.111988,58.857688
4,1.198116,8.557975,67.415663
5,1.045365,7.466890,74.882553


In [6]:
fa, loadings_rotated, eigen_rot, expl_rot, cum_rot = run_rotated_factor_analysis(X, n_factors)

factors_after = create_factor_summary_df(eigen_rot, expl_rot, cum_rot)
print("Факторы после вращения:")
display(factors_after)

Факторы после вращения:


,Собственное значение,% объясненной дисперсии,Кумулятивный % объясненной дисперсии
1,3.688872,26.349087,26.349087
2,1.982349,14.159637,40.508725
3,1.883695,13.454966,53.963691
4,1.513512,10.810800,64.774490
5,1.415129,10.108063,74.882553


**Вращение факторов (кватримакс) приводит к более равномерному распределению объясненной дисперсии между факторами и к увеличению удельного веса каждого из них (за исключением первого фактора).
Вращение факторов также позволяет достичь более простой структуры факторных нагрузок для исходных переменных, что может несколько упростить интерпретацию полученных факторов.**

In [7]:
loadings_pca = pca.components_.T * np.sqrt(pca.explained_variance_)

loadings_df = pd.DataFrame(
    loadings_pca[:, :n_factors],
    index=X.columns,
    columns=[f'F{i+1}' for i in range(n_factors)]
)

loadings_rotated_df = pd.DataFrame(
    loadings_rotated,
    index=X.columns,
    columns=[f'F{i+1}' for i in range(n_factors)]
)

loadings_filtered = filter_loadings(loadings_df)
loadings_rotated_filtered = filter_loadings(loadings_rotated_df)

print("Матрица факторных нагрузок до вращения")
display(loadings_filtered)

print("\nМатрица факторных нагрузок после вращения")
display(loadings_rotated_filtered)

Матрица факторных нагрузок до вращения


,F1,F2,F3,F4,F5
k1,0.791,,,,
k2,0.716,,,,
k4,0.801,,,,
k5,0.643,-0.621,,,
k6,,,,0.478,0.602
k7,0.543,,,,0.459
k9,0.563,-0.631,,,
k10,,0.741,,,
k11,,,,0.613,-0.488
k13,,0.573,0.486,,



Матрица факторных нагрузок после вращения


,F1,F2,F3,F4,F5
k1,0.911,,,,
k2,0.760,,,,
k4,0.901,,,,
k5,0.654,0.616,,,
k6,,,,,0.876
k7,,,,,0.723
k9,0.529,0.685,,,
k10,,-0.872,,,
k11,,,,0.853,
k13,,,0.826,,


**В интерпретации учитываются нагрузки, абсолютное значение которых превышает 0.4,
что соответствует умеренной и высокой связи переменной с фактором.**

## Расчет интегрального показателя

In [8]:
factor_scores = fa.transform(X)

factor_scores_df = pd.DataFrame(
    factor_scores,
    index=X.index,
    columns=[f'F{i+1}' for i in range(n_factors)]
)

I = calculate_integral_indicator(factor_scores_df, expl_rot)
factor_scores_df.insert(0, "Integral Indicator", I)

display(factor_scores_df.round(3))

,Integral Indicator,F1,F2,F3,F4,F5
0,-0.159,-0.315,0.829,-1.069,-0.026,-0.460
1,0.424,1.043,1.147,-0.178,0.022,0.081
2,-0.333,-0.613,1.103,-0.857,-0.730,-1.320
3,0.801,2.530,-0.508,1.009,0.124,0.568
4,-0.168,0.809,0.460,-1.647,-0.808,-1.352
...,...,...,...,...,...,...
2690,0.334,0.773,-0.264,0.868,1.245,-0.827
2691,0.330,-0.115,1.253,0.647,0.525,0.390
2692,0.173,-0.099,0.455,1.861,0.616,-1.804
2693,0.182,-0.812,1.149,0.794,0.600,0.613


**Полученный интегральный показатель может быть использован для ранжирования предприятий
по уровню кредитоспособности и применяется в последующих лабораторных работах.**